## Version 2

### 2 Convolutions / leaky_relu / 0.2 Dropouts / 20 epochs

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical, image_dataset_from_directory
from sklearn.metrics import confusion_matrix

In [10]:
train_path = 'Kaggle_MRI_Tumor/Training'
test_path = 'Kaggle_MRI_Tumor/Testing'

classes = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']

train_dataset = image_dataset_from_directory(directory=train_path,
                                             label_mode='categorical',
                                             class_names=classes)
test_dataset = image_dataset_from_directory(directory=test_path,
                                            label_mode='categorical',
                                            class_names=classes)

Found 2870 files belonging to 4 classes.
Found 394 files belonging to 4 classes.


In [11]:
model = tf.keras.models.Sequential()

# Preprocessing 
model.add(layers.Resizing(height=128, width=128, interpolation='bilinear'))
model.add(layers.Rescaling(scale=1./255))

# Data Augmentation
model.add(layers.RandomFlip(mode='horizontal_and_vertical'))
model.add(layers.RandomRotation(factor=.2))

In [12]:
# First Convolution
model.add(layers.Conv2D(filters=64, kernel_size=(1,5)))
model.add(layers.Conv2D(filters=64, kernel_size=(5,1)))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Dropout(.2))

In [13]:
# Second Convolution
model.add(layers.Conv2D(filters=64, kernel_size=(1,3)))
model.add(layers.Conv2D(filters=64, kernel_size=(3,1)))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Dropout(.2))

In [14]:
# Feed forward
model.add(layers.Flatten())
model.add(layers.Dense(512))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(.2))
model.add(layers.Dense(512))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(.2))
model.add(layers.Dense(4))
model.add(layers.Activation('softmax'))

In [ ]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001, decay=1e-6)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_dataset, epochs=20, validation_data=test_dataset)

In [ ]:
model.evaluate(test_dataset)

In [ ]:
model.summary()